In [1]:
import os
import sys

repo_root_path = os.path.abspath(os.path.join(os.path.pardir, 'hebbnets'))
if repo_root_path not in sys.path:
    sys.path.append(repo_root_path)

import matplotlib.pyplot as plt
%matplotlib nbagg

import numpy as np

from demo_utils import yeild_patch_batch, turn_input_weights_to_pilimg, place_pilimgs_in_grid
from hebbnets.hebbnets.hah_model import MultilayerHahNetwork

# Make image patch generator

In [2]:
image_urls = [
    'https://upload.wikimedia.org/wikipedia/commons/3/39/GlarusAlps.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/3/38/M_Rainier.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/3/3c/Volc%C3%A1n_Chimborazo%2C_%22El_Taita_Chimborazo%22.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/0/06/Briesetal_bei_Briese.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/7/75/Florida_freshwater_swamp_usgov_image.jpg'
]

patch_size = [12, 12]

batch_o_patches_gen = yeild_patch_batch(
    image_urls,
    patch_size=patch_size,
    patch_stride=[9, 9],
    batch_size=5000
)

# Gather/whiten all patches

In [3]:
# Whitening image patches

all_patches = np.vstack([
    np.array([p.ravel() / 255.0 for p in batch])
    for batch in batch_o_patches_gen
])

all_patches -= all_patches.ravel().mean()
all_patches /= all_patches.ravel().std()

cov_mat = all_patches.T.dot(all_patches) / all_patches.shape[0]
whiten_mat = np.linalg.pinv(cov_mat + 0.01 * np.eye(cov_mat.shape[0]))
dewhiten_mat = np.linalg.pinv(whiten_mat)
all_patches_white = all_patches.dot(whiten_mat)

# Set up HAH network

In [4]:
input_layer_size = patch_size[0] * patch_size[1]
nodes_per_layer = [128]

hah_network = MultilayerHahNetwork(
    input_layer_size,
    nodes_per_layer,
    has_bias=False,
    act_type='soft_thresh',
    reg_lambda=2.0,
    noise_var=0.001
)

# Train network on batches of patches

In [ ]:
hah_network.train(
    all_patches_white,
    num_epochs=1
)

/Users/mcmenamin/GitHub/hebbnets/hebbnets/layers.py:234: RuntimeWarning: overflow encountered in square
  activation_sqr = self.activation ** 2
/Users/mcmenamin/GitHub/hebbnets/hebbnets/layers.py:244: RuntimeWarning: invalid value encountered in true_divide
  activation_sqr_norm = activation_sqr / self._cum_sqr_activation
/Users/mcmenamin/.virtualenvs/py3env/lib/python3.5/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)
/Users/mcmenamin/.virtualenvs/py3env/lib/python3.5/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/Users/mcmenamin/GitHub/hebbnets/hebbnets/layers.py:43: RuntimeWarning: invalid value encountered in maximum
  abs_thresh = np.maximum(np.abs(activation) - self._soft_thresh_val, 0.0)


# Visualize the learned recpetive fields

In [ ]:
rf_picture = place_pilimgs_in_grid(
    turn_input_weights_to_pilimg(
        hah_network.layers[0],
        patch_size,
        dewhiten_mat
    )
)

In [ ]:
hah_network.layers[0]._soft_thresh_val

In [ ]:
img_rescale = 3.0

rf_picture.resize(
    [int(img_rescale * i) for i in rf_picture.size],
)